# **Installations:**

In [1]:
!pip install -q nltk pycocoevalcap
!pip install -q nltk pandas
!pip install -q sacrebleu
!pip install -q bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 771.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 935.9 kB/s eta 0:00:00


In [11]:
import sacrebleu
import pandas as pd
import ast
import nltk
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from bert_score import score
import math


### Data preparation:

In [7]:
model_name = "blip" #@param ["gpt", "blip"]

if your model_name is blip , Choose ground truth answers, corresponding to your data

In [3]:
op1 = "gold_weird_questions_weird_images.csv"
op2 = "gold_normal_questions_weird_images.csv"
op3 = "gold_normal_questions_normal_images.csv"

gold_df = pd.read_csv(op2)

input_file_for_calculation.csv is not a real file, it can be any file you want to check it's scores

In [5]:
input_file_for_calculation = "strange_answers_gpt_prompt_no explain_blip.csv" #@param {type:"string"}

In [8]:
# Ensure you have the necessary NLTK data files
nltk.download('punkt')

# Load data from CSV
df = pd.read_csv(input_file_for_calculation)

# Function to extract the embedded answer
def extract_embedded_answer(question):
    try:
        # Convert string representation of list to actual list
        question_list = ast.literal_eval(question)
        # print(question_list)
        if isinstance(question_list, list) and len(question_list) > 1:
            return question_list[1]
        else:
            raise ValueError("Question does not have the expected format.")
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing question: {question} - {e}")
        return None

# Apply function to extract the embedded answer
if model_name == "gpt":
  df['embedded_answer'] = df['question'].apply(extract_embedded_answer)
elif model_name == "blip":
  df['embedded_answer'] = gold_df['gold answers']
# Filter out rows where parsing failed
df = df.dropna(subset=['embedded_answer'])

# Prepare the data for evaluation
gts = {idx: [row['answer']] for idx, row in df.iterrows()}
res = {idx: [row['embedded_answer']] for idx, row in df.iterrows()}
references =df['answer'].tolist()
candidates = df['embedded_answer'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Checking the expected data
for i in candidates:
  print(i)

print(len(references))
print(len(candidates))

### Calculate CIDEr score:

In [ ]:
# Initialize the evaluators
scorers = [
    (Cider(), "CIDEr"),
]

all_results_dict = {}

# Calculate the scores
for scorer, method in scorers:
    print(f'Computing {method} score...')
    score, scores = scorer.compute_score(gts, res)
    if isinstance(score, list):
        all_results_dict[method] = round(score[3] * 100, 2)
        print(f"{method}: {round(score[3] * 100, 2)}\n")
    else:
        all_results_dict[method] = round(score * 100, 2)
        print(f"{method}: {round(score * 100, 2)}\n")

# Print the overall results
print(all_results_dict)

### Calculate SacreBLEU score:

In [12]:
bleu_scores = []

# Calculate SacreBLEU score for each pair
for i in range(len(references)):
    if references[i] is None or isinstance(references[i], float) and math.isnan(references[i]):
      bleu_scores.append(0.0)
    else:
      # sacrebleu expects a list of references
      reference = [references[i]]
      hypothesis = candidates[i]

      bleu = sacrebleu.sentence_bleu(hypothesis, reference)
      bleu_scores.append(bleu.score)

sumblue = 0
for i in bleu_scores:
  sumblue += float(i)

# Calculate the average BLEU score
average_bleu = sumblue / len(bleu_scores)
print(f"Average SacreBLEU score: {average_bleu:.2f}")

Average SacreBLEU score: 0.00


## Calculate Bert score:

### Calculate bert score with a simple example:

In [16]:
# Define the reference and candidate sentences
references = ["ocean"]
candidates = ["sea"]

# Calculate BERTScore
P, R, F1 = score(candidates, references, lang="en", verbose=True)

# Print the scores
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall: {R.mean().item():.4f}")
print(f"F1: {F1.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.28 seconds, 3.53 sentences/sec
Precision: 0.9980
Recall: 0.9980
F1: 0.9980


### Calculate bert score for the given file:

In [13]:
from ast import literal_eval  # To safely evaluate the string representation of lists

# Compute BERTScore for each row
bert_scores = []
sum_bert = 0
for index, row in df.iterrows():
    actual_answer = row['answer']
    embedded_answer = row['embedded_answer']

    if isinstance(actual_answer, float) and math.isnan(actual_answer):
      bert_scores.append(0.0)  # Assign the lowest score (0.0)
      sum_bert += 0.0
    else:
      P, R, F1 = score([actual_answer], [embedded_answer], lang='en', model_type='bert-base-uncased')
      bert_scores.append(F1.item())
      sum_bert += F1.item()

average_bert_score = sum_bert / len(bert_scores)
print(f"Average BERTScore: {average_bert_score:.4f}")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Average BERTScore: 0.0040
